## ASA Week 09: Topic Modeling and Word Embedding in Gensim


Part 1. Topic Modeling using Gensim
---

Let's first take a look at how to perform topic modeling using the Python package [*Gensim*](https://radimrehurek.com/gensim/).

We need ot install the following two packages:

- pip install gensim
- pip install pyldavis

We will use the [movie review dataset](https://www.kaggle.com/c/word2vec-nlp-tutorial/data) in the file "reviews.csv" as the training data (each line is a review document).

In [14]:
# read textual review data
lines = open("reviews.csv",encoding="utf-8",mode="r").readlines()
reviews = [line.strip().split(',')[1] for line in lines]
print(reviews[:2])

[' with all this stuff going down at the moment with mj started listening to his music watching the odd documentary here and there watched the wiz and watched moonwalker again maybe just want to get certain insight into this guy who thought wa really cool in the eighty just to maybe make up my mind whether he is guilty or innocent moonwalker is part biography part feature film which remember going to see at the cinema when it wa originally released some of it ha subtle message about mj feeling towards the press and also the obvious message of drug are bad visually impressive but of course this is all about michael jackson so unless you remotely like mj in anyway then you are going to hate this and find it boring some may call mj an egotist for consenting to the making of this movie but mj and most of his fan would say that he made it for the fan which if true is really nice of him the actual feature film bit when it finally start is only on for 20 minute or so excluding the smooth crim

In [3]:
# process data
from gensim.parsing.preprocessing import remove_stopwords

# remove stopwords
reviews = [remove_stopwords(review) for review in reviews]
# review doc: string to list
reviews = [review.split() for review in reviews]

In [4]:
# prepare data for gensim
from gensim import corpora

%time vocabs = corpora.Dictionary(reviews) # generate vocabulary dictionary
vocabs.save('movie_reviews.dict') # you can save the dictionary in disk for future use
print(vocabs)

%time corpus = [vocabs.doc2bow(review_doc) for review_doc in reviews] # generate corpus for training model
corpora.MmCorpus.serialize('movie_reviews.mm', corpus) # you can save the corpus for future use
print(corpus[0])

CPU times: user 2.92 s, sys: 29.8 ms, total: 2.95 s
Wall time: 3 s
Dictionary(65049 unique tokens: ['20', 'actual', 'attention', 'away', 'bad']...)
CPU times: user 1.82 s, sys: 85 ms, total: 1.91 s
Wall time: 1.98 s
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 3), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 2), (28, 1), (29, 1), (30, 1), (31, 3), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 2), (39, 2), (40, 1), (41, 2), (42, 1), (43, 1), (44, 1), (45, 1), (46, 3), (47, 1), (48, 2), (49, 2), (50, 1), (51, 3), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 2), (59, 2), (60, 1), (61, 1), (62, 3), (63, 1), (64, 1), (65, 1), (66, 3), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 3), (73, 3), (74, 2), (75, 1), (76, 1), (77, 11), (78, 1), (79, 2), (80, 3), (81, 2), (82, 1), (83, 2), (84, 1), (85, 1), (86, 1), (87, 1), 

In [5]:
# train LDA model
from gensim import models

%time lda = models.ldamodel.LdaModel(corpus=corpus, id2word=vocabs, num_topics=100)
lda.save('reviews_lda_100.model') # you can save the trained model for future use

# you can load the saved model later as follows:
# lda = models.LdaModel.load('reviews_lda_100.model')

CPU times: user 1min 22s, sys: 7.25 s, total: 1min 29s
Wall time: 52.6 s


In [6]:
# show topics
lda.show_topics(-1) # print topics

[(0,
  '0.032*"steve" + 0.026*"robot" + 0.021*"dan" + 0.020*"fantasy" + 0.014*"machine" + 0.011*"life" + 0.011*"renaissance" + 0.011*"human" + 0.010*"grandmother" + 0.010*"real"'),
 (1,
  '0.045*"film" + 0.011*"horror" + 0.009*"wa" + 0.008*"character" + 0.008*"director" + 0.007*"ha" + 0.006*"doe" + 0.006*"like" + 0.006*"time" + 0.005*"good"'),
 (2,
  '0.106*"thomas" + 0.085*"reunion" + 0.059*"overlooked" + 0.059*"kurt" + 0.036*"photograph" + 0.035*"sheet" + 0.030*"marty" + 0.029*"blowing" + 0.028*"prank" + 0.024*"ada"'),
 (3,
  '0.059*"jane" + 0.055*"animal" + 0.053*"tarzan" + 0.039*"native" + 0.035*"jungle" + 0.034*"ape" + 0.033*"chris" + 0.018*"elephant" + 0.017*"walken" + 0.015*"toilet"'),
 (4,
  '0.046*"comedy" + 0.027*"funny" + 0.015*"film" + 0.014*"laugh" + 0.012*"humour" + 0.011*"great" + 0.011*"hilarious" + 0.011*"adam" + 0.011*"humor" + 0.009*"funniest"'),
 (5,
  '0.052*"fight" + 0.050*"art" + 0.042*"action" + 0.019*"martial" + 0.016*"fighting" + 0.015*"master" + 0.013*"chines

In [7]:
# represent documents using topic distributions
corpus_lda = lda[corpus]
print(corpus_lda[0])

[(6, 0.22105563), (11, 0.22829233), (23, 0.023068018), (55, 0.1682661), (60, 0.05279133), (69, 0.18127695), (80, 0.012877543), (86, 0.034553945), (95, 0.018431295)]


In [8]:
# visualize the trained lda model using pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models # pyLDAvis version = 3.3.x
# import pyLDAvis.gensim # pyLDAvis version = 3.2.2
pyLDAvis.enable_notebook()

%time vis_data = pyLDAvis.gensim_models.prepare(lda, corpus, vocabs) # pyLDAvis version = 3.3.x
# %time vis_data = pyLDAvis.gensim_models.prepare(lda, corpus, vocabs) # pyLDAvis version = 3.2.2

CPU times: user 1min 22s, sys: 6.4 s, total: 1min 28s
Wall time: 1min 32s


In [9]:
vis_data

/Users/gavinshi/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
55    -0.178606  0.100385       1        1  13.337824
60    -0.179282  0.130141       2        1   8.393578
91    -0.150051 -0.072166       3        1   4.218272
11    -0.151965  0.122118       4        1   3.697154
76    -0.151934 -0.066265       5        1   3.544808
...         ...       ...     ...      ...        ...
32     0.178835  0.028232      96        1   0.177064
51     0.184847 -0.042179      97        1   0.160318
94     0.214674  0.011642      98        1   0.159799
84     0.309752  0.020811      99        1   0.135967
2      0.358930  0.031084     100        1   0.069998

[100 rows x 5 columns], topic_info=         Term          Freq         Total  Category  logprob  loglift
126        wa  53778.000000  53778.000000   Default  30.0000  30.0000
80      movie  54614.000000  54614.000000   Default  29.0000  29.0000
41       film  49033.000000  49033.000000   Default  28.0000  28.0000
1014   horror   3759.000000   3759.000000   Default  27.0000  27.0000
2298  episode   2635.000000   2635.000000   Default  26.0000  26.0000
...       ...           ...           ...       ...      ...      ...
1089   matrix      4.668232    206.159149  Topic100  -5.9537   3.4766
914    school     24.548209   1657.780228  Topic100  -4.2938   3.0519
183     agent      4.070400    469.878636  Topic100  -6.0907   2.5157
250     group      5.508276   1170.054300  Topic100  -5.7882   1.9059
254      high      5.170516   2063.176457  Topic100  -5.8515   1.2754

[6539 rows x 6 columns], token_table=       Topic      Freq    Term
term                          
1627       1  0.243265      10
1627       2  0.170821      10
1627       3  0.017288      10
1627       4  0.107020      10
1627       5  0.034164      10
...      ...       ...     ...
12989     20  0.976928     zoo
2291      88  0.967040    zoom
12057      2  0.026334      zu
12057     12  0.948026      zu
1672      37  0.925365  zuniga

[45899 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[56, 61, 92, 12, 77, 40, 39, 22, 20, 38, 26, 31, 14, 32, 91, 35, 81, 87, 75, 97, 74, 7, 2, 5, 16, 90, 49, 43, 93, 54, 63, 44, 23, 100, 55, 45, 98, 60, 15, 69, 64, 82, 46, 6, 53, 65, 57, 78, 42, 70, 80, 27, 17, 99, 30, 34, 8, 13, 21, 1, 84, 24, 88, 19, 89, 29, 28, 94, 83, 41, 62, 11, 79, 76, 10, 36, 50, 59, 68, 9, 96, 86, 73, 58, 25, 37, 71, 72, 66, 51, 4, 48, 47, 67, 18, 33, 52, 95, 85, 3])

### Exercise

In this exercise, please use Gensim to train a LDA model with *50* topics on the above movie review dataset, and visualize the trained model using pyLDAvis. Can you get more meaningful topics compared with the previous LDA model with 100 topics?

In [10]:
# write your code here

/Users/gavinshi/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
lines = open("reviews.csv",encoding="utf-8",mode="r").readlines()
reviews = [line.strip().split(',')[1] for line in lines]

# remove stopwords
reviews = [remove_stopwords(review) for review in reviews]
# review doc: string to list
reviews = [review.split() for review in reviews]

%time vocabs = corpora.Dictionary(reviews) # generate vocabulary dictionary
%time corpus = [vocabs.doc2bow(review_doc) for review_doc in reviews] # generate corpus for training model

%time lda = models.ldamodel.LdaModel(corpus=corpus, id2word=vocabs, num_topics=50)

vis_data = pyLDAvis.gensim_models.prepare(lda, corpus, vocabs)

vis_data

/Users/gavinshi/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


CPU times: user 3.67 s, sys: 24.4 ms, total: 3.7 s
Wall time: 3.85 s
CPU times: user 2.17 s, sys: 58.6 ms, total: 2.23 s
Wall time: 2.25 s
CPU times: user 1min 1s, sys: 4.96 s, total: 1min 6s
Wall time: 40.4 s


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
20     0.173156  0.091471       1        1  13.765281
21     0.138943 -0.063380       2        1   8.258352
6      0.135152 -0.042982       3        1   7.290090
5      0.147557  0.179748       4        1   5.868676
16     0.152592  0.038256       5        1   5.734791
39     0.128352 -0.086961       6        1   5.424001
24     0.137645  0.052217       7        1   5.140410
22     0.134225 -0.004778       8        1   4.944664
12     0.130244 -0.042477       9        1   4.253500
37     0.115640  0.063142      10        1   2.926572
46     0.083302  0.034423      11        1   2.632434
44     0.096473 -0.024538      12        1   2.565381
7      0.111981  0.124044      13        1   2.298575
0      0.081721 -0.102480      14        1   1.832534
9      0.075311 -0.119047      15        1   1.820516
36     0.092225 -0.014601      16        1   1.791233
4      0.079020 -0.079518      17        1   1.767929
8      0.057027 -0.028277      18        1   1.698065
26     0.060460  0.125154      19        1   1.307964
48     0.052797  0.055563      20        1   1.205310
13     0.059493 -0.015239      21        1   1.178494
14     0.053379 -0.063933      22        1   1.168655
47     0.058616  0.031099      23        1   1.132337
10     0.009321 -0.043386      24        1   1.043703
43     0.030109 -0.085883      25        1   0.949312
42     0.040301 -0.026769      26        1   0.915416
17     0.028910 -0.086434      27        1   0.870520
27     0.031791 -0.005101      28        1   0.869853
33     0.001672 -0.037541      29        1   0.785393
38    -0.001146 -0.056321      30        1   0.704004
11    -0.008931  0.038591      31        1   0.664773
23     0.004664  0.036168      32        1   0.632571
28    -0.012811 -0.026725      33        1   0.630354
49    -0.022372 -0.011698      34        1   0.544102
2     -0.100440 -0.081347      35        1   0.500722
31    -0.055520  0.030076      36        1   0.491893
32    -0.035887  0.023827      37        1   0.491083
25    -0.045181 -0.014460      38        1   0.453422
18    -0.107058 -0.004483      39        1   0.410680
29    -0.094984  0.019778      40        1   0.409274
30    -0.146989  0.018116      41        1   0.401293
40    -0.116382  0.220793      42        1   0.316314
41    -0.181443 -0.004400      43        1   0.285420
35    -0.191163 -0.002157      44        1   0.280932
15    -0.226374 -0.042314      45        1   0.275340
3     -0.226772  0.019871      46        1   0.273078
19    -0.194031  0.012028      47        1   0.219248
45    -0.225995 -0.000268      48        1   0.214890
34    -0.244811  0.001332      49        1   0.186405
1     -0.263792  0.001799      50        1   0.174240, topic_info=             Term          Freq         Total Category  logprob  loglift
80          movie  54339.000000  54339.000000  Default  30.0000  30.0000
41           film  48396.000000  48396.000000  Default  29.0000  29.0000
126            wa  52581.000000  52581.000000  Default  28.0000  28.0000
1120       series   3452.000000   3452.000000  Default  27.0000  27.0000
2298      episode   2621.000000   2621.000000  Default  26.0000  26.0000
...           ...           ...           ...      ...      ...      ...
11267  motorcycle     35.869473     61.831481  Topic50  -4.8266   5.8080
21247      hudson     50.409465    111.495652  Topic50  -4.4863   5.5587
12300      lavish     30.166199     55.079468  Topic50  -4.9997   5.7504
2559      dorothy     33.535701    110.235663  Topic50  -4.8938   5.1625
6440    melodrama     36.617807    214.004438  Topic50  -4.8059   4.5870

[3528 rows x 6 columns], token_table=       Topic      Freq  Term
term                        
1627       1  0.281793    10
1627       2  0.024539    10
1627       3  0.021676    10
1627       4  0.206539    10
1627       5  0.052350    10
...      ...       ...   ...
447

Part 2. Word Embedding using Gensim
---

Now, let's take a look at how to train word embedding model using Gensim. We will also use the [movie review dataset](https://www.kaggle.com/c/word2vec-nlp-tutorial/data). When preparing the data for training Word2Vec model, we process the textual review data as a collection of sentences rathen than documents. This is a standard routine which usually results in better model. The processed data is in the file "review_sentences.csv" (each line is a review sentence).


In [13]:
# read textual review data

review_sents = open("review_sentences.csv",encoding="utf-8",mode="r").readlines()
review_sents = [sent.strip().split() for sent in review_sents]
print(len(review_sents))
print(review_sents[:2])

/Users/gavinshi/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


218725
[['stuff', 'going', 'moment', 'mj', 'started', 'listening', 'music', 'watching', 'odd', 'documentary', 'watched', 'wiz', 'watched', 'moonwalker'], ['maybe', 'want', 'certain', 'insight', 'guy', 'thought', 'wa', 'cool', 'eighty', 'maybe', 'mind', 'guilty', 'innocent']]


Different from training topic models in Gensim, we do not need to build the dictionary and corpus by ourselves when training a Word2Vec model.

In [14]:
# train word2vec skipgram model
from gensim.models import Word2Vec

# set model parameters
num_features = 100    # Word vector dimensionality                      
min_word_count = 5    # Minimum word count                        
context = 5           # Context window size
sg = 1                # skipgram=1, cbow=0
num_workers = 4       # Number of threads to run in parallel

# train the model
%time model_sg100 = Word2Vec(review_sents, vector_size=num_features, window=context, min_count=min_word_count, sg=sg, workers=num_workers)

# you can save the trained model for future use
model_sg100.save("reviews_word2vec_skipgram_100.model")

# you can load saved model later as follows:
# model_w2v = Word2Vec.load("reviews_word2vec_skipgram_100.model")

/Users/gavinshi/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


CPU times: user 1min 52s, sys: 917 ms, total: 1min 53s
Wall time: 45.6 s


For more details about the Word2Vec model parameters, you can refer to the [Gensim documentation](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec).

Once the model is trained, we can examine it as follows:

In [15]:
# examine the trained word2vec skipgram model

print('embedding vector of the word "movie":')
print(model_sg100.wv['movie']) # word embedding vector of the word "movie"
print(model_sg100.wv.most_similar(positive=['woman', 'king'], negative=['man'])) # woman-man+king=?
print(model_sg100.wv.doesnt_match("man woman child kitchen".split()))
print(model_sg100.wv.doesnt_match("france england germany berlin".split()))
print(model_sg100.wv.doesnt_match("breakfast cereal dinner lunch".split()))
print(model_sg100.wv.most_similar("man"))
print(model_sg100.wv.most_similar("woman"))
print(model_sg100.wv.most_similar("awful"))
print(model_sg100.wv.most_similar("excellent"))
print(model_sg100.wv.most_similar("cage"))

embedding vector of the word "movie":
[-0.13262676  0.16056094  0.28365612  0.22836727  0.040193   -0.1073219
  0.19703384  0.20268294 -0.33878648 -0.18812129 -0.21795915 -0.41134977
  0.23389016  0.178141    0.10468085 -0.2573494   0.70758843 -0.15889019
 -0.21894109 -0.29213256  0.27027076  0.1569035   0.23812015 -0.33200762
  0.1388619  -0.02589815 -0.2270016   0.32956994 -0.37776718  0.16968004
  0.28669795 -0.18214868 -0.13263872 -0.42392448 -0.1299315  -0.21997157
  0.15610188  0.2358365  -0.08900692 -0.00480238  0.23624206 -0.35871303
 -0.06051981  0.08753847  0.0051061   0.02259411 -0.03256884  0.03449161
  0.091901    0.25280926 -0.05675744 -0.29356095 -0.30331823 -0.40025184
 -0.305623    0.06302369 -0.00423238  0.01069803 -0.12573676  0.01525086
 -0.18297216 -0.28861108  0.08750325  0.22940835 -0.01921438  0.5184546
 -0.04013998  0.09056639 -0.38322476  0.5284049   0.07507122 -0.10726035
  0.5324626  -0.02726773  0.00198081  0.22666588  0.3940536   0.04165513
 -0.10894325  0

/Users/gavinshi/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
# all word embeddings
word_vectors = model_sg100.wv.vectors
print(type(word_vectors))
print(word_vectors.shape) # num_words * num_features

<class 'numpy.ndarray'>
(24137, 100)


/Users/gavinshi/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### We can use  [gensim.models.phrases](https://radimrehurek.com/gensim/models/phrases.html#module-gensim.models.phrases) module for phrases with more than one word.

In [17]:
# train a bigram word2vec skipgram model
from gensim.models import Phrases
from gensim.models.phrases import Phraser

bigram_transformer = Phraser(Phrases(review_sents))
%time model_sg100_bigram = Word2Vec(bigram_transformer[review_sents], vector_size=100, window=5, min_count=5, sg=1, workers=4)
model_sg100_bigram.save("reviews_word2vec_skipgram_100_bigram.model")

/Users/gavinshi/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


CPU times: user 2min 10s, sys: 1 s, total: 2min 11s
Wall time: 51.3 s


In [18]:
# now we can play with some bigram phrases
print(model_sg100_bigram.wv.most_similar("nicolas_cage"))
print(model_sg100_bigram.wv.most_similar("harry_potter"))

[('paul_newman', 0.935321033000946), ('quinn', 0.9315627217292786), ('olsen', 0.9299476146697998), ('chris_cooper', 0.9235896468162537), ('kurt', 0.9209234714508057), ('william_macy', 0.9201018810272217), ('tony_danza', 0.9188805222511292), ('dreyfus', 0.9177072048187256), ('sam_neill', 0.9170575141906738), ('micheal', 0.9167933464050293)]
[('talespin', 0.9301210641860962), ('undoubtedly_best', 0.9263731837272644), ('zu_warrior', 0.9237160086631775), ('pale_comparison', 0.9185023903846741), ('goodfellas', 0.9182243347167969), ('disney_animated', 0.9170673489570618), ('stayed_true', 0.9166109561920166), ('house_wax', 0.9149521589279175), ('ado', 0.9144208431243896), ('mary_poppins', 0.9143704771995544)]


/Users/gavinshi/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Exercise

In this exercise, please use Gensim to train a Word2Vec CBOW (rather than Skipgram) model on the above movie review dataset (i.e., the file "review_sentences.csv") as follows:

- set the model parameters as follows:
    * num_features = 100 (word vector dimension)
    * min_count = 5 (minimum word count)
    * window = 5 (context window size)
    * sg = 0 (train a **CBOW** model)
- play with the learnt model by examining:
    * the most similar words to some words, e.g., excellent, cage
    * find the word that does not match with other words in a word list
- qualitatively compare the CBOW model with Skipgram model, which is better in your opinion?

In [19]:
# write your code here

/Users/gavinshi/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [20]:
review_sents = open("review_sentences.csv",encoding="utf-8",mode="r").readlines()
review_sents = [sent.strip().split() for sent in review_sents]

# set model parameters
num_features = 100    # Word vector dimensionality                      
min_word_count = 5    # Minimum word count                        
context = 5           # Context window size
sg = 0               # skipgram=1, cbow=0
num_workers = 4       # Number of threads to run in parallel

# train the model
%time model_cbow = Word2Vec(review_sents, vector_size=num_features, window=context, min_count=min_word_count, sg=sg, workers=num_workers)

/Users/gavinshi/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


CPU times: user 39.8 s, sys: 201 ms, total: 40 s
Wall time: 14.3 s


In [21]:
print(model_cbow.wv.most_similar("excellent"))
print(model_cbow.wv.most_similar("cage"))

[('fantastic', 0.8971798419952393), ('outstanding', 0.8954408168792725), ('fine', 0.877120316028595), ('superb', 0.8752452731132507), ('terrific', 0.8617680668830872), ('solid', 0.8468698859214783), ('brilliant', 0.8424371480941772), ('wonderful', 0.8410651087760925), ('exceptional', 0.8381034731864929), ('fabulous', 0.8247731328010559)]
[('hopper', 0.8688058853149414), ('wilson', 0.8406587243080139), ('goldblum', 0.8380635976791382), ('phillips', 0.8287017941474915), ('bacon', 0.8267509341239929), ('dern', 0.826496422290802), ('nicolas', 0.8223949670791626), ('jeff', 0.818874180316925), ('nicholas', 0.8181320428848267), ('freeman', 0.8176431655883789)]


/Users/gavinshi/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
print(model_cbow.wv.doesnt_match(['breakfast', 'cereal', 'dinner', 'lunch']))

dinner


/Users/gavinshi/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
# Skip-gram model is better

/Users/gavinshi/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Homework (Optional): Topic Modeling and Word Embedding on 10-K MD&A Corpus
---
In this homework, we will use another dataset - the MD&A texts (item 7) of 10-K forms in 2012. For your convenience, I have already processed the data in the file "10kmda2012_sentences.csv" (each line is a MD&A sentence).

**Task 1:** train a LDA model on the processed 10-K MD&A corpus:
- only use the first 100,000 sentences in the corpus (otherwise, it will cost too much time)
- set the number of topics to 100
- visualize the trained model using pyLDAvis
- manually check whether the learnt topics are meaningful

**Task 2:** train a Word2Vec model on the processed 10K MD&A corpus:
- set the model parameters as follows:
    + num_features = 100 (word vector dimension)
    + min_count = 5 (minimum word count)
    + window = 5 (context window size)
    + sg = 1 (i.e., train a **skip-gram** model)
- play with the learnt model by examining:
    + the most similar words to some words, e.g., apple, ceo, excellent
    + find the word that does not match with other words in a word list, e.g., "apple google amazon banana", ""breakfast cereal dinner lunch""

In [24]:
# write your code here

/Users/gavinshi/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
# LDA
sents = open("10kmda2012_sentences.csv",encoding="utf-8",mode="r").readlines()
sents = [sent.strip() for sent in sents]
sents = sents[:100000]

/Users/gavinshi/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [26]:
# process data
from gensim.parsing.preprocessing import remove_stopwords

# remove stopwords
sents = [remove_stopwords(sent) for sent in sents]
# review doc: string to list
sents = [sent.split() for sent in sents]

/Users/gavinshi/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
# prepare data for gensim
from gensim import corpora

%time vocabs = corpora.Dictionary(sents) # generate vocabulary dictionary
# vocabs.save('movie_reviews.dict') # you can save the dictionary in disk for future use
print(vocabs)

%time corpus = [vocabs.doc2bow(sent_doc) for sent_doc in sents] # generate corpus for training model
# corpora.MmCorpus.serialize('movie_reviews.mm', corpus) # you can save the corpus for future use
print(corpus[0])

/Users/gavinshi/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


CPU times: user 2.62 s, sys: 15.4 ms, total: 2.64 s
Wall time: 2.66 s
Dictionary(17151 unique tokens: ['analysis', 'condition', 'discussion', 'financial', 'management']...)
CPU times: user 1.97 s, sys: 64.4 ms, total: 2.03 s
Wall time: 2.04 s
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)]


In [28]:
# train LDA model
from gensim import models

%time lda = models.ldamodel.LdaModel(corpus=corpus, id2word=vocabs, num_topics=100)

/Users/gavinshi/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


CPU times: user 50.4 s, sys: 1.55 s, total: 52 s
Wall time: 49.1 s


In [29]:
# visualize the trained lda model using pyLDAvis
pyLDAvis.enable_notebook()
import pyLDAvis
import pyLDAvis.gensim_models # pyLDAvis version = 3.3.x

%time vis_data = pyLDAvis.gensim_models.prepare(lda, corpus, vocabs) # pyLDAvis version = 3.3.x
vis_data

/Users/gavinshi/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


CPU times: user 32.4 s, sys: 1.4 s, total: 33.8 s
Wall time: 38.6 s


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
78     0.324034 -0.094615       1        1  3.355931
98     0.168708  0.014144       2        1  2.566021
65     0.121632  0.084505       3        1  2.228419
18     0.278475 -0.058134       4        1  2.093467
99     0.038201  0.203548       5        1  2.001741
...         ...       ...     ...      ...       ...
7     -0.034160  0.016290      96        1  0.494786
3      0.007653  0.100677      97        1  0.489273
66    -0.042827  0.003444      98        1  0.488350
70    -0.054725 -0.026028      99        1  0.458502
45     0.047150 -0.037878     100        1  0.347031

[100 rows x 5 columns], topic_info=               Term          Freq         Total  Category  logprob  loglift
163         million  28544.000000  28544.000000   Default  30.0000  30.0000
1831           loan  13503.000000  13503.000000   Default  29.0000  29.0000
56             sale  13457.000000  13457.000000   Default  28.0000  28.0000
71             rate  13458.000000  13458.000000   Default  27.0000  27.0000
374         company  11330.000000  11330.000000   Default  26.0000  26.0000
...             ...           ...           ...       ...      ...      ...
2449  restructuring      0.078711   1454.921472  Topic100 -11.1686  -4.1612
163         million      0.109631  28544.104119  Topic100 -10.8373  -6.8063
176        increase      0.072997  11331.029251  Topic100 -11.2440  -6.2891
102            cost      0.058873  13278.551044  Topic100 -11.4590  -6.6628
173        customer      0.055249   5770.011043  Topic100 -11.5226  -5.8928

[3636 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
2463     38  0.891660        aa
2400     69  0.789332    abbott
384      15  0.997128   ability
384      22  0.000462   ability
384      96  0.001387   ability
...     ...       ...       ...
617      51  0.999225     yield
1193     87  0.988879  yielding
2875     67  0.996858      york
1926     39  0.084085      zero
1926     58  0.905529      zero

[4213 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[79, 99, 66, 19, 100, 1, 73, 35, 56, 27, 86, 18, 45, 39, 7, 52, 77, 12, 51, 98, 70, 60, 57, 81, 74, 36, 75, 85, 11, 13, 65, 80, 92, 17, 96, 41, 72, 64, 20, 43, 24, 58, 28, 55, 31, 9, 88, 63, 48, 97, 29, 69, 54, 21, 42, 89, 5, 93, 91, 22, 68, 23, 49, 6, 62, 37, 87, 16, 95, 83, 84, 25, 47, 26, 40, 3, 44, 76, 32, 14, 82, 33, 10, 50, 30, 2, 94, 15, 53, 78, 90, 38, 34, 59, 61, 8, 4, 67, 71, 46])

In [30]:
# word2vec

/Users/gavinshi/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [31]:
sents = open("10kmda2012_sentences.csv",encoding="utf-8",mode="r").readlines()
sents = [sent.strip().split() for sent in sents]

/Users/gavinshi/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [32]:
# train word2vec skipgram model
from gensim.models import Word2Vec

# set model parameters
num_features = 100    # Word vector dimensionality                      
min_word_count = 5    # Minimum word count                        
context = 5           # Context window size
sg = 1                # skipgram=1, cbow=0
num_workers = 4       # Number of threads to run in parallel

# train the model
%time model_sg = Word2Vec(review_sents, vector_size=num_features, window=context, min_count=min_word_count, sg=sg, workers=num_workers)

/Users/gavinshi/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


CPU times: user 1min 48s, sys: 615 ms, total: 1min 49s
Wall time: 32.8 s


In [33]:
print(model_sg.wv.most_similar("apple"))
print(model_sg.wv.most_similar("ceo"))
print(model_sg.wv.most_similar("excellent"))

[('bicycle', 0.7788037061691284), ('fish', 0.7763434648513794), ('cone', 0.7683797478675842), ('bowl', 0.7620484828948975), ('shiny', 0.7609529495239258), ('sensation', 0.7603657841682434), ('straddling', 0.7584481239318848), ('pond', 0.7582493424415588), ('mud', 0.7581585645675659), ('lebowski', 0.7571097016334534)]
[('trautman', 0.9422417879104614), ('alphonse', 0.9385595917701721), ('rushton', 0.9356604218482971), ('commender', 0.9348047971725464), ('inefficient', 0.9346885085105896), ('murdock', 0.9346219897270203), ('considine', 0.9336346983909607), ('magnate', 0.9326514005661011), ('assan', 0.93124920129776), ('organizing', 0.9298204183578491)]
[('superb', 0.8397080302238464), ('outstanding', 0.8107491135597229), ('terrific', 0.7982479333877563), ('fantastic', 0.7792264223098755), ('fine', 0.7720721960067749), ('marvelous', 0.7704530358314514), ('great', 0.7698812484741211), ('stellar', 0.7666733264923096), ('exceptional', 0.7586615085601807), ('brilliant', 0.756466269493103)]


/Users/gavinshi/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [34]:
print(model_sg.wv.doesnt_match(['apple', 'google', 'amazon', 'banana']))
print(model_sg.wv.doesnt_match(['breakfast', 'cereal', 'dinner', 'lunch']))

apple
dinner


/Users/gavinshi/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
